In [1]:
from azureml.core import Workspace, Experiment

# Configure experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='cifar10_cnn_remote')

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Cluster configuration
cluster_name = "aml-cpu"
min_nodes = 0
max_nodes = 2
vm_size = "STANDARD_D2_V2"

# Verify that the cluster exists already
try:
    aml_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print('Cluster not `%s` not found, creating one now.' % cluster_name)
    config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=min_nodes, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=config)

# Wait until the cluster is ready
aml_cluster.wait_for_completion(show_output=True)

Cluster not `aml-cpu` not found, creating one now.
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [4]:
## from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core import RunConfiguration

# Create a remote run configuration
run_amlcompute = RunConfiguration()

run_amlcompute.target = aml_cluster
run_amlcompute.environment.docker.enabled = True
run_amlcompute.environment.docker.base_image = DEFAULT_CPU_IMAGE

run_amlcompute.environment.python.user_managed_dependencies = False
run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-sdk[notebooks,contrib,automl,explain]',
    'azureml-tensorboard',
    'tensorflow==1.13.2',
    'keras==2.3.0'
])

In [5]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig

script = 'cifar10_cnn_remote.py'
script_folder = os.path.join(os.getcwd(), 'code')

src = ScriptRunConfig(source_directory=script_folder, script=script, run_config=run_amlcompute)
run = exp.submit(src)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…